In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

# 读取数据
data = pd.read_excel('UTS.xlsx')

# 分割数据
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=73)

# 最佳参数
best_params = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.1,
    'max_depth': 5,
    'min_child_weight': 5.411729501146951,
    'n_estimators': 139,
    'reg_alpha': 1.0,
    'reg_lambda': 1.0,
    'subsample': 0.5,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# 使用最佳参数训练最终模型
UTS_model = xgb.XGBRegressor(**best_params)
UTS_model.fit(x_train, y_train)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# 读取数据
data = pd.read_excel('EL.xlsx')

# 分割数据
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=16)

# 最佳参数
best_params = {
    'bootstrap': False,
    'max_depth': 10,
    'max_features': 0.5,
    'min_impurity_decrease': 0.0,
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'n_estimators': 55
}

# 使用最佳参数训练最终模型
EL_model = RandomForestRegressor(**best_params, random_state=25)
EL_model.fit(x_train, y_train)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

# 读取数据
data = pd.read_excel('HV.xlsx')

# 分割数据
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=75)

# 最佳参数
best_params = {
    'colsample_bytree': 0.7537221430165258,
    'learning_rate': 0.07141712725507017,
    'max_depth': 3,
    'min_child_weight': 1.036994365622379,
    'n_estimators': 179,
    'reg_alpha': 0.07844930740270828,
    'reg_lambda': 0.4659923785137676,
    'subsample': 0.5124079817369307,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# 使用最佳参数训练最终模型
Hardness_model = xgb.XGBRegressor(**best_params)
Hardness_model.fit(x_train, y_train)



In [ ]:
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

#

# 目标值
target_UTS = 300  # 设定目标UTS值
target_EL = 3.5    # 设定目标EL值
target_Hardness = 120  # 设定目标Hardness值

# 权重
weight_UTS = 1  # UTS的权重
weight_EL = 1   # EL的权重
weight_Hardness = 1  # Hardness的权重

# 定义目标函数
def target_function(**params):
    params_df = pd.DataFrame([params], columns=X.columns)  # 保证特征顺序一致
    pred_UTS = UTS_model.predict(params_df)[0]
    pred_EL = EL_model.predict(params_df)[0]
    pred_Hardness = Hardness_model.predict(params_df)[0]
    
    # 计算目标值：三个预测性能与目标性能的比值之和
    ratio_UTS = pred_UTS / target_UTS
    ratio_EL = pred_EL / target_EL
    ratio_Hardness = pred_Hardness / target_Hardness
    
    score =weight_UTS*abs(ratio_UTS - 1) + weight_EL*abs(ratio_EL - 1) + weight_Hardness*abs(ratio_Hardness - 1)
    
    return -score  # 优化器会最大化目标函数，所以这里返回负值以最小化差值

# 定义特征的容许范围
pbounds = {
    'Al': (3.0, 8.0),  # 例如铝的范围从0到10
    'Cu': (0.0, 6.0),   # 例如铜的范围从0到5
    'Mg': (0.0, 0.05),   # 例如镁的范围从0到5
    'Ti': (0.0, 0.0),   # 例如钛的范围从0到5
    'Mn': (0.0, 0.0),   # 例如锰的范围从0到5
    'Si': (0.0, 0.0),   # 例如硅的范围从0到5
    'La': (0.0, 0.0),   # 例如镧的范围从0到5
    'Ce': (0.0, 0.0)    # 例如铈的范围从0到5
}

# 贝叶斯优化
optimizer = BayesianOptimization(
    f=target_function,
    pbounds=pbounds,
    random_state=1
)

# 最大迭代次数
n_iter = 50

# 运行贝叶斯优化
optimizer.maximize(init_points=10, n_iter=n_iter)

# 获取最佳参数
best_params = optimizer.max['params']
best_params_df = pd.DataFrame([best_params], columns=X.columns)  # 保证特征顺序一致
best_pred_UTS = UTS_model.predict(best_params_df)[0]
best_pred_EL = EL_model.predict(best_params_df)[0]
best_pred_Hardness = Hardness_model.predict(best_params_df)[0]

# 容忍误差
tolerance_UTS = 20  # 可以根据需要调整
tolerance_EL = 0.5  # 可以根据需要调整
tolerance_Hardness = 10 # 可以根据需要调整

# 检查是否在容忍误差范围内
if (abs(best_pred_UTS - target_UTS) <= tolerance_UTS and
    abs(best_pred_EL - target_EL) <= tolerance_EL and
    abs(best_pred_Hardness - target_Hardness) <= tolerance_Hardness):
    print(f"找到的最佳参数: {best_params}")
    print(f"预测的UTS值: {best_pred_UTS}")
    print(f"预测的EL值: {best_pred_EL}")
    print(f"预测的Hardness值: {best_pred_Hardness}")
else:
    print("在指定迭代次数内未找到满足条件的解。")


